# BB84 with Proper Coherent States - Jupyter Notebook Implementation

## Mathematical Background

### Coherent States
A coherent state $|\alpha\rangle$ is defined as:
$$|\alpha\rangle = e^{-|\alpha|^2/2} \sum_{n=0}^{\infty} \frac{\alpha^n}{\sqrt{n!}} |n\rangle$$

### Photon Number Distribution
For weak coherent states with average photon number $\mu = |\alpha|^2$:
$$P(n) = \frac{\mu^n e^{-\mu}}{n!}$$

---

## Alice's Preparation Phase

Alice needs to:
1. Choose random bits and bases
2. Encode each bit+basis combination as a coherent state amplitude
3. Create proper coherent state representations (not circuits!)

---

# Bob measurement phase

## Sift key and analyze

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import math
from collections import Counter
from typing import List, Tuple, Dict, Optional, Union
from dataclasses import dataclass


@dataclass
class WCSPulse:
    """Represents a weak coherent state pulse"""
    bit: int
    basis: int
    photon_count: int
    mu: float
    intensity_label: str = "signal"


@dataclass
class BB84Results:
    """Container for BB84 protocol results"""
    alice_key: List[int]
    bob_key: List[int]
    qber: float
    key_rate: float
    detection_rate: float
    vacuum_rate: float
    sifted_bits: int
    total_pulses: int
    intensity_distribution: Dict[str, int]


@dataclass
class PNSAnalysisResult:
    """Container for PNS attack analysis results"""
    is_secure: bool
    security_margin: float
    signal_yield: float
    decoy_yields: Dict[str, float]
    estimated_multiphoton_yield: float
    security_condition_lhs: float
    security_condition_rhs: float
    analysis_details: Dict[str, float]


class DecoyStateQKD:
    """
    BB84 Quantum Key Distribution with Weak Coherent States and Decoy State Protocol
    
    This implementation models realistic QKD systems using:
    - Weak coherent states with Poisson photon statistics
    - Decoy state protocol for security against photon-number-splitting attacks
    - PNS attack detection based on Hwang's 2003 method
    - Realistic detection losses and technical errors
    """
    
    def __init__(self, 
                 mu_signal: float = 0.1,
                 mu_decoy: List[float] = None,
                 decoy_probabilities: List[float] = None,
                 detection_efficiency: float = 1.0,
                 technical_error_rate: float = 0.02,
                 random_seed: Optional[int] = None):
        """
        Initialize the QKD system parameters
        
        Args:
            mu_signal: Average photon number for signal states
            mu_decoy: List of average photon numbers for decoy states
            decoy_probabilities: Probabilities for using each decoy state
            detection_efficiency: Single-photon detector efficiency
            technical_error_rate: Technical error rate in measurements
            random_seed: Random seed for reproducibility
        """
        self.mu_signal = mu_signal
        self.mu_decoy = mu_decoy if mu_decoy is not None else [0.05, 0.0]
        self.decoy_probabilities = decoy_probabilities if decoy_probabilities is not None else [0.3, 0.1]
        self.detection_efficiency = detection_efficiency
        self.technical_error_rate = technical_error_rate
        
        # Validate inputs
        if len(self.mu_decoy) != len(self.decoy_probabilities):
            raise ValueError("mu_decoy and decoy_probabilities must have same length")
        if sum(self.decoy_probabilities) >= 1:
            raise ValueError("Sum of decoy probabilities must be less than 1")
        
        self.signal_probability = 1 - sum(self.decoy_probabilities)
        
        if random_seed is not None:
            random.seed(random_seed)
            np.random.seed(random_seed)
    
    def poisson_probability(self, n: int, mu: float) -> float:
        """
        Calculate Poisson probability P_n(μ) from Hwang's paper Eq. before (2)
        P_n(μ) = e^(-μ) * μ^n / n!
        """
        return np.exp(-mu) * (mu ** n) / math.factorial(n)
    
    def sample_photon_number(self, mu: float) -> int:
        """Sample photon number from Poisson distribution"""
        return np.random.poisson(mu)
    
    def create_wcs_pulse(self, bit: int, basis: int, mu: float, intensity_label: str = "signal") -> WCSPulse:
        """
        Create a weak coherent state pulse
        
        Args:
            bit: Information bit (0 or 1)
            basis: Encoding basis (0=Z-basis, 1=X-basis)
            mu: Average photon number
            intensity_label: Label for intensity type
        
        Returns:
            WCSPulse object
        """
        n_photons = self.sample_photon_number(mu)
        return WCSPulse(bit=bit, basis=basis, photon_count=n_photons, 
                       mu=mu, intensity_label=intensity_label)
    
    def alice_prepare_pulses(self, bits: List[int], bases: List[int], 
                           verbose: bool = False) -> Tuple[List[WCSPulse], List[str]]:
        """
        Alice prepares WCS pulses with decoy states
        
        Args:
            bits: List of bits to encode
            bases: List of bases to use
            verbose: Print detailed information
        
        Returns:
            Tuple of (pulses, intensity_labels)
        """
        if verbose:
            print("=== Alice prepares WCS pulses with decoy states ===")
            print(f"Signal intensity: μ_s = {self.mu_signal:.3f} (probability: {self.signal_probability:.3f})")
            for i, (mu_d, prob_d) in enumerate(zip(self.mu_decoy, self.decoy_probabilities)):
                print(f"Decoy {i+1} intensity: μ_d{i+1} = {mu_d:.3f} (probability: {prob_d:.3f})")
        
        pulses = []
        intensity_labels = []
        intensity_stats = {'signal': {'count': 0, 'photons': []}}
        
        for i, mu_d in enumerate(self.mu_decoy):
            intensity_stats[f'decoy_{i}'] = {'count': 0, 'photons': []}
        
        for i, (bit, basis) in tqdm(enumerate(zip(bits, bases)), total=len(bits), desc="Alice preparing"):
            # Choose intensity randomly
            rand = random.random()
            cumulative_prob = 0
            chosen_mu = self.mu_signal
            intensity_label = 'signal'
            
            for j, prob_d in enumerate(self.decoy_probabilities):
                cumulative_prob += prob_d
                if rand < cumulative_prob:
                    chosen_mu = self.mu_decoy[j]
                    intensity_label = f'decoy_{j}'
                    break
            
            pulse = self.create_wcs_pulse(bit, basis, chosen_mu, intensity_label)
            pulses.append(pulse)
            intensity_labels.append(intensity_label)
            
            # Update statistics
            intensity_stats[intensity_label]['count'] += 1
            intensity_stats[intensity_label]['photons'].append(pulse.photon_count)
            
            if verbose and i < 8:
                basis_name = "Z-basis" if basis == 0 else "X-basis"
                state_name = {(0,0): "|0⟩", (1,0): "|1⟩", (0,1): "|+⟩", (1,1): "|-⟩"}[(bit, basis)]
                print(f"Pulse {i}: bit={bit} → {state_name} ({basis_name}), "
                      f"μ={chosen_mu:.3f} ({intensity_label}), n={pulse.photon_count} photons")
        
        if verbose:
            self._print_intensity_statistics(intensity_stats, len(bits))
        
        return pulses, intensity_labels
    
    def _print_intensity_statistics(self, intensity_stats: Dict, total_pulses: int):
        """Print intensity distribution statistics"""
        print(f"\nIntensity distribution:")
        for label, stats in intensity_stats.items():
            count = stats['count']
            actual_prob = count / total_pulses
            avg_photons = np.mean(stats['photons']) if stats['photons'] else 0
            print(f"  {label}: {count} pulses ({actual_prob:.1%}), avg_photons={avg_photons:.3f}")
    
    def bb84_measurement_outcome(self, alice_bit: int, alice_basis: int, bob_basis: int) -> int:
        """
        Simulate BB84 measurement outcome
        
        Returns Bob's measurement result based on BB84 rules:
        - Same basis: Alice's bit (with small error probability)
        - Different basis: Random result
        """
        if alice_basis == bob_basis:
            # Matching bases → correct result (with technical error)
            if random.random() < self.technical_error_rate:
                return 1 - alice_bit  # Bit flip error
            else:
                return alice_bit
        else:
            # Mismatched bases → random result
            return random.randint(0, 1)
    
    def bob_measure_pulses(self, pulses: List[WCSPulse], bob_bases: List[int], 
                          eavesdropping: bool = False, verbose: bool = False) -> Tuple[List[Optional[int]], int, int]:
        """
        Bob measures WCS pulses using single-photon detectors
        
        Args:
            pulses: List of WCS pulses from Alice
            bob_bases: Bob's measurement bases
            eavesdropping: Whether Eve is present (reduces photon count via PNS attack)
            verbose: Print detailed information
        
        Returns:
            Tuple of (measurement_results, detected_count, vacuum_count)
        """
        if verbose:
            print("=== Bob measures WCS pulses ===")
        
        results = []
        detected_count = 0
        vacuum_count = 0
        
        cnt = []
        
        for i, (pulse, bob_basis) in tqdm(enumerate(zip(pulses, bob_bases)), total=len(pulses), desc="Bob measuring"):
            n_photons = pulse.photon_count
            
            # Simulate eavesdropping (Eve performs PNS attack)
            # Eve blocks single-photon pulses and steals one photon from multiphoton pulses
            if eavesdropping and n_photons >= 2:
                n_photons_received = n_photons - 1  # Eve steals one photon
            elif eavesdropping and n_photons == 1:
                n_photons_received = 0  # Eve blocks single-photon pulses
            else:
                n_photons_received = n_photons
            cnt.append(n_photons_received)
            
            if n_photons_received == 0:
                # Vacuum pulse - no detection possible
                result = None
                vacuum_count += 1
                detection_status = "vacuum - no detection"
            else:
                # Attempt detection with efficiency η
                # Detection probability: 1 - (1-η)^n (from standard detector model)
                detection_prob = 1 - (1 - self.detection_efficiency) ** n_photons_received
                
                if random.random() > detection_prob:
                    result = None
                    detection_status = f"{n_photons_received} photons - detection failed"
                else:
                    # Successful detection - perform BB84 measurement
                    detected_count += 1
                    result = self.bb84_measurement_outcome(pulse.bit, pulse.basis, bob_basis)
                    detection_status = f"{n_photons_received} photons - detected bit {result}"
            
            results.append(result)
            
            if verbose and i < 8:
                basis_name = "Z-basis" if bob_basis == 0 else "X-basis"
                print(f"Pulse {i}: {basis_name} → {detection_status}")
        
        if verbose:
            total_pulses = len(pulses)
            detection_rate = detected_count / total_pulses
            vacuum_rate = vacuum_count / total_pulses
            print(f"\nDetection summary:")
            print(f"  Total pulses: {total_pulses}")
            print(f"  Vacuum pulses: {vacuum_count} ({vacuum_rate:.1%})")
            print(f"  Successful detections: {detected_count} ({detection_rate:.1%})")
            
        if verbose:
            print("Received photon count distribution:", Counter(cnt))
        
        return results, detected_count, vacuum_count
    
    def sift_keys(self, alice_bits: List[int], alice_bases: List[int], 
                  bob_bases: List[int], bob_results: List[Optional[int]], 
                  intensity_labels: List[str], verbose: bool = False) -> Tuple[List[int], List[int], List[int], List[str]]:
        """
        Perform basis sifting to extract the shared key
        
        Returns:
            Tuple of (alice_key, bob_key, matching_indices, sifted_intensity_labels)
        """
        if verbose:
            print("=== Basis sifting ===")
        
        # Find matching bases with successful detection
        matching_indices = []
        for i in range(len(alice_bases)):
            if alice_bases[i] == bob_bases[i] and bob_results[i] is not None:
                matching_indices.append(i)
        
        # Extract sifted data
        alice_sifted = [alice_bits[i] for i in matching_indices]
        bob_sifted = [bob_results[i] for i in matching_indices]
        sifted_intensity_labels = [intensity_labels[i] for i in matching_indices]
        
        # Extract signal state bits for the key
        signal_indices = [i for i, label in enumerate(sifted_intensity_labels) if label == 'signal']
        alice_key = [alice_sifted[i] for i in signal_indices]
        bob_key = [bob_sifted[i] for i in signal_indices]
        
        if verbose:
            print(f"Total sifted bits: {len(alice_sifted)} (all intensities)")
            print(f"Signal state key: {len(alice_key)} bits")
            print(f"Sifted intensity distribution: {Counter(sifted_intensity_labels)}")
        
        return alice_key, bob_key, matching_indices, sifted_intensity_labels
    
    def calculate_qber(self, alice_key: List[int], bob_key: List[int]) -> float:
        """Calculate Quantum Bit Error Rate"""
        if not alice_key:
            return 0.0
        errors = sum(1 for a, b in zip(alice_key, bob_key) if a != b)
        return errors / len(alice_key)
    
    def calculate_yields(self, pulses: List[WCSPulse], bob_results: List[Optional[int]], 
                        intensity_labels: List[str], alice_bases: List[int], 
                        bob_bases: List[int]) -> Dict[str, float]:
        """
        Calculate yields Y_s and Y_d from Hwang's paper Eq. (2)
        
        Y_s = Σ_n P_n(μ) y_n        (signal yield)
        Y_d = Σ_n P_n(μ') y_n'      (decoy yield)
        
        Where y_n is the probability that n-photon pulse is detected
        """
        yields = {}
        
        # Group data by intensity
        intensity_data = {}
        for i, (pulse, result, label, alice_basis, bob_basis) in enumerate(
            zip(pulses, bob_results, intensity_labels, alice_bases, bob_bases)):
            
            # Only consider matching bases (sifted data)
            if alice_basis != bob_basis:
                continue
                
            if label not in intensity_data:
                intensity_data[label] = {'total': 0, 'detected': 0}
            
            intensity_data[label]['total'] += 1
            if result is not None:  # Detection occurred
                intensity_data[label]['detected'] += 1
        
        # Calculate yields for each intensity
        for label, data in intensity_data.items():
            if data['total'] > 0:
                yields[label] = data['detected'] / data['total']
            else:
                yields[label] = 0.0
        
        return yields
    
    def analyze_pns_attack(self, pulses: List[WCSPulse], bob_results: List[Optional[int]], 
                          intensity_labels: List[str], alice_bases: List[int], 
                          bob_bases: List[int], verbose: bool = True) -> PNSAnalysisResult:
        """
        Analyze if channel is compromised by PNS attack using Hwang's decoy state method
        
        Based on Hwang's 2003 paper security condition Eq. (12):
        Y_s > (P_2(μ) / P_2(μ')) * Y_d
        
        Where:
        - Y_s: signal yield
        - Y_d: decoy yield  
        - P_2(μ): probability of 2-photon pulse from signal source
        - P_2(μ'): probability of 2-photon pulse from decoy source
        
        Returns:
            PNSAnalysisResult with security analysis
        """
        if verbose:
            print("=== PNS Attack Detection Analysis ===")
            print("Based on Hwang's 2003 decoy state method")
        
        # Calculate yields Y_s and Y_d from Eq. (2)
        yields = self.calculate_yields(pulses, bob_results, intensity_labels, 
                                     alice_bases, bob_bases)
        
        if verbose:
            print(f"\nMeasured yields:")
            for label, yield_val in yields.items():
                if label == 'signal':
                    print(f"  Y_s (signal): {yield_val:.4f}")
                else:
                    print(f"  Y_d ({label}): {yield_val:.4f}")
        
        # Get signal and primary decoy yields
        Y_s = yields.get('signal', 0.0)
        
        # Use the first non-vacuum decoy state for analysis
        decoy_yields = {k: v for k, v in yields.items() if k.startswith('decoy_')}
        
        # Find the decoy state with highest μ for primary analysis
        primary_decoy_label = None
        primary_decoy_mu = 0
        for i, mu_d in enumerate(self.mu_decoy):
            label = f'decoy_{i}'
            if label in decoy_yields and mu_d > primary_decoy_mu:
                primary_decoy_mu = mu_d
                primary_decoy_label = label
        
        if primary_decoy_label is None:
            # Fallback to any available decoy
            primary_decoy_label = list(decoy_yields.keys())[0] if decoy_yields else None
            primary_decoy_mu = self.mu_decoy[0] if self.mu_decoy else 0.1
        
        Y_d = decoy_yields.get(primary_decoy_label, 0.0) if primary_decoy_label else 0.0
        
        # Calculate P_2(μ) and P_2(μ') from Poisson distribution
        # P_2(μ) = e^(-μ) * μ^2 / 2! 
        P_2_signal = self.poisson_probability(2, self.mu_signal)
        P_2_decoy = self.poisson_probability(2, primary_decoy_mu)
        
        if verbose:
            print(f"\nPoisson probabilities (2-photon):")
            print(f"  P_2(μ_s={self.mu_signal:.3f}) = {P_2_signal:.6f}")
            print(f"  P_2(μ_d={primary_decoy_mu:.3f}) = {P_2_decoy:.6f}")
        
        # Calculate security condition from Eq. (12): Y_s > (P_2(μ)/P_2(μ')) * Y_d
        if P_2_decoy > 0:
            security_ratio = P_2_signal / P_2_decoy
            security_condition_rhs = security_ratio * Y_d
            security_margin = Y_s - security_condition_rhs
            is_secure = security_margin > 0
        else:
            security_ratio = float('inf')
            security_condition_rhs = float('inf')
            security_margin = -float('inf')
            is_secure = False
        
        # Estimate multiphoton yield bound from Eq. (9): Y_s^m ≤ (P_2(μ)/P_2(μ')) * Y_d
        estimated_multiphoton_yield = security_condition_rhs
        
        if verbose:
            print(f"\nSecurity Analysis (Hwang's Eq. 12):")
            print(f"  Security condition: Y_s > (P_2(μ)/P_2(μ')) * Y_d")
            print(f"  Left side (Y_s): {Y_s:.6f}")
            print(f"  Right side: ({P_2_signal:.6f}/{P_2_decoy:.6f}) * {Y_d:.6f} = {security_condition_rhs:.6f}")
            print(f"  Security margin: {security_margin:.6f}")
            print(f"  Channel status: {'SECURE' if is_secure else 'COMPROMISED'}")
            
            if not is_secure:
                print(f"  ⚠️  WARNING: Possible PNS attack detected!")
                print(f"     Decoy yield is abnormally high compared to signal yield")
            else:
                print(f"  ✅ Channel appears secure against PNS attacks")
        
        # Additional analysis details
        analysis_details = {
            'P_2_signal': P_2_signal,
            'P_2_decoy': P_2_decoy,
            'security_ratio': security_ratio,
            'primary_decoy_mu': primary_decoy_mu,
            'primary_decoy_label': primary_decoy_label
        }
        
        return PNSAnalysisResult(
            is_secure=is_secure,
            security_margin=security_margin,
            signal_yield=Y_s,
            decoy_yields=decoy_yields,
            estimated_multiphoton_yield=estimated_multiphoton_yield,
            security_condition_lhs=Y_s,
            security_condition_rhs=security_condition_rhs,
            analysis_details=analysis_details
        )
    
    def run_security_comparison(self, bits_length: int = 100000, verbose: bool = True) -> Dict[str, PNSAnalysisResult]:
        """
        Run protocol with and without eavesdropping to demonstrate PNS detection
        
        Returns:
            Dictionary with analysis results for both scenarios
        """
        if verbose:
            print("=== Security Comparison: Clean Channel vs PNS Attack ===")
        
        # Generate test data
        alice_bits = np.random.randint(0, 2, bits_length)
        alice_bases = np.random.randint(0, 2, bits_length)
        bob_bases = np.random.randint(0, 2, bits_length)
        
        results = {}
        
        for scenario in ['clean_channel', 'pns_attack']:
            eavesdropping = (scenario == 'pns_attack')
            
            if verbose:
                print(f"\n--- {scenario.replace('_', ' ').title()} ---")
            
            # Alice prepares pulses
            wcs_pulses, intensity_labels = self.alice_prepare_pulses(
                alice_bits, alice_bases, verbose=False
            )
            
            # Bob measures (with or without eavesdropping)
            bob_results, detected_pulses, vacuum_pulses = self.bob_measure_pulses(
                wcs_pulses, bob_bases, eavesdropping=eavesdropping, verbose=False
            )
            
            # Analyze for PNS attack
            analysis = self.analyze_pns_attack(
                wcs_pulses, bob_results, intensity_labels, 
                alice_bases, bob_bases, verbose=verbose
            )
            
            results[scenario] = analysis
        
        return results
    
    def create_bb84_results(self, alice_key: List[int], bob_key: List[int], 
                           detected_count: int, vacuum_count: int, 
                           total_pulses: int, intensity_labels: List[str],
                           sifted_intensity_labels: List[str]) -> BB84Results:
        """
        Create BB84Results object from protocol data
        
        Args:
            alice_key: Alice's final key
            bob_key: Bob's final key  
            detected_count: Number of detected pulses
            vacuum_count: Number of vacuum pulses
            total_pulses: Total number of pulses sent
            intensity_labels: All intensity labels
            sifted_intensity_labels: Intensity labels after sifting
        
        Returns:
            BB84Results object
        """
        qber = self.calculate_qber(alice_key, bob_key)
        key_rate = len(alice_key) / total_pulses
        detection_rate = detected_count / total_pulses
        vacuum_rate = vacuum_count / total_pulses
        
        return BB84Results(
            alice_key=alice_key,
            bob_key=bob_key,
            qber=qber,
            key_rate=key_rate,
            detection_rate=detection_rate,
            vacuum_rate=vacuum_rate,
            sifted_bits=len(sifted_intensity_labels),
            total_pulses=total_pulses,
            intensity_distribution=Counter(intensity_labels)
        )

# Decoy State Quantum Key Distribution: Mathematical Framework

## Table of Contents
1. [Introduction](#introduction)
2. [The Photon-Number-Splitting Attack Problem](#the-photon-number-splitting-attack-problem)
3. [Weak Coherent States and Poisson Statistics](#weak-coherent-states-and-poisson-statistics)
4. [The Decoy State Solution](#the-decoy-state-solution)
5. [Mathematical Analysis of Eve's Attack](#mathematical-analysis-of-eves-attack)
6. [Security Conditions](#security-conditions)
7. [Implementation in Code](#implementation-in-code)

---

## Introduction

The decoy state protocol, proposed by Won-Young Hwang in 2003, provides a solution to the photon-number-splitting (PNS) attack in quantum key distribution (QKD) systems using weak coherent states. This document explains the complete mathematical framework from first principles.

## The Photon-Number-Splitting Attack Problem

### Traditional BB84 Security Issue

In ideal BB84, Alice sends perfect single photons. However, practical implementations use **weak coherent states** which sometimes contain multiple photons. This creates a vulnerability:

**Traditional Security Condition:**
$$y > P_{\text{multi}}$$

Where:
- $y = 1 - l$ is the channel yield (transmission probability)
- $l$ is the channel loss
- $P_{\text{multi}}$ is the probability of multiphoton pulses

**The Problem:** For high-loss channels (satellite communications), $y$ becomes very small, requiring nearly perfect single-photon sources.

### Photon-Number-Splitting Attack

Eve's attack strategy:
1. **Measure photon number** $n$ non-destructively
2. **If $n = 1$:** Block the pulse completely
3. **If $n \geq 2$:** Split the pulse, keep one photon, send the rest to Bob
4. **Later:** Measure her stored photons in the correct basis (announced publicly)

This attack is "invisible" because Bob still sees the expected detection rate, but Eve gains full information about multiphoton pulses.

---

## Weak Coherent States and Poisson Statistics

### Coherent State Definition

A coherent state $|\alpha\rangle$ with complex amplitude $\alpha$ can be written as:

$$|\alpha\rangle = e^{-|\alpha|^2/2} \sum_{n=0}^{\infty} \frac{\alpha^n}{\sqrt{n!}} |n\rangle$$

### Phase Randomization

For QKD security, Alice randomizes the phase $\theta$ of her coherent states $|\mu e^{i\theta}\rangle$. After averaging over random phases:

$$\rho = \int \frac{d\theta}{2\pi} |\mu e^{i\theta}\rangle\langle\mu e^{i\theta}| = \sum_{n=0}^{\infty} P_n(\mu) |n\rangle\langle n|$$

### Poisson Photon Statistics

The probability of having exactly $n$ photons follows a Poisson distribution:

$$P_n(\mu) = \frac{\mu^n e^{-\mu}}{n!}$$

Where $\mu = |\alpha|^2$ is the **average photon number**.

**Key Properties:**
- $\sum_{n=0}^{\infty} P_n(\mu) = 1$ (normalization)
- $\langle n \rangle = \mu$ (mean)
- $\text{Var}(n) = \mu$ (variance)

---

## The Decoy State Solution

### Multi-Intensity Strategy

Alice uses **multiple photon sources** with different intensities:

1. **Signal Source $S$:** Average photon number $\mu < 1$ (mostly single photons)
2. **Decoy Source $S'$:** Average photon number $\mu' \geq 1$ (mostly multiphotons)

### Random Intensity Selection

For each pulse, Alice randomly chooses:
- Signal intensity with probability $1 - \alpha$
- Decoy intensity with probability $\alpha$

**Crucial Point:** Eve cannot distinguish multiphoton pulses from signal vs. decoy sources based on photon number alone.

### Yield Definitions

The **$n$-photon yield** $y_n$ is the probability that an $n$-photon pulse is detected by Bob:

$$y_n = \text{Pr}(\text{detection} | n \text{ photons})$$

**Total yields** for each source:

$$Y_s = \sum_{n=0}^{\infty} P_n(\mu) y_n \quad \text{(Signal yield)}$$

$$Y_d = \sum_{n=0}^{\infty} P_n(\mu') y_n' \quad \text{(Decoy yield)}$$

**Multiphoton yield** from signal source:

$$Y_s^m = \sum_{n=2}^{\infty} P_n(\mu) y_n$$

**Normalized multiphoton yield:**

$$\tilde{Y}_s^m = \frac{\sum_{n=2}^{\infty} P_n(\mu) y_n}{\sum_{n=2}^{\infty} P_n(\mu)}$$

---

## Mathematical Analysis of Eve's Attack

### Indistinguishability Constraint

**Key Assumption:** Eve cannot distinguish multiphoton pulses from different sources:

$$y_n = y_n' \quad \text{for all } n$$

This means Eve's strategy for $n$-photon pulses must be the same regardless of source.

### Eve's Optimization Problem

Eve wants to maximize the ratio:

$$A = \frac{\sum_{n=2}^{\infty} P_n(\mu) y_n}{\sum_{n=2}^{\infty} P_n(\mu') y_n}$$

Subject to the constraint that Bob observes the expected total yields.

### Mathematical Bound on Eve's Strategy

For $\mu < \mu'$, we have:

$$\frac{P_n(\mu)}{P_n(\mu')} > \frac{P_m(\mu)}{P_m(\mu')} \quad \text{if } n < m$$

**Proof:**
$$\frac{P_n(\mu)}{P_n(\mu')} = \frac{e^{-\mu}\mu^n/n!}{e^{-\mu'}(\mu')^n/n!} = e^{\mu'-\mu} \left(\frac{\mu}{\mu'}\right)^n$$

Since $\mu < \mu'$, we have $\mu/\mu' < 1$, so $(\mu/\mu')^n$ decreases as $n$ increases.

### Eve's Optimal Strategy

**Theorem:** Eve's optimal strategy satisfies:

$$A \leq \frac{P_2(\mu)}{P_2(\mu')}$$

**Proof:** Using the ordering property above:

$$\frac{P_2(\mu)}{P_2(\mu')} - A = \frac{1}{P_2(\mu') \sum_{n=2}^{\infty} P_n(\mu) y_n} \sum_{n=2}^{\infty} y_n [P_2(\mu)P_n(\mu') - P_2(\mu')P_n(\mu)] \geq 0$$

**Eve's Optimal Choice:** Set $y_2 > 0$ and $y_n = 0$ for $n \geq 3$.

**Physical Interpretation:** Eve blocks all pulses with more than 2 photons, since they're more likely to come from the decoy source.

---

## Security Conditions

### Upper Bound on Signal Multiphoton Yield

From the constraint $\sum_{n=2}^{\infty} P_n(\mu') y_n \leq Y_d$ and Eve's optimal strategy:

$$Y_s^m \leq \frac{P_2(\mu)}{P_2(\mu')} Y_d$$

### Security Condition (Hwang's Main Result)

For the protocol to be secure, the total detected signals must exceed the attacked multiphotons:

$$Y_s > \left[\sum_{n=2}^{\infty} P_n(\mu)\right] \tilde{Y}_s^m$$

Substituting the bound from above:

$$\boxed{Y_s > \frac{P_2(\mu)}{P_2(\mu')} Y_d}$$

**This is the core security condition for decoy state QKD.**

### Loss-Independent Security

In normal operation (no attack), $Y_d \approx \frac{\mu'}{\mu} Y_s$, so the condition becomes:

$$Y_s > \frac{P_2(\mu)}{P_2(\mu')} \cdot \frac{\mu'}{\mu} Y_s$$

Simplifying:

$$1 > \frac{P_2(\mu)}{P_2(\mu')} \cdot \frac{\mu'}{\mu} = \frac{e^{\mu'}\mu}{e^{\mu}\mu'}$$

**Key Insight:** This condition is **independent of channel loss** $l$, unlike the traditional condition $y > P_{\text{multi}}$.

### Numerical Example

For $\mu = 0.3$ and $\mu' = 1.0$:

$$P_2(0.3) = e^{-0.3} \frac{(0.3)^2}{2!} \approx 0.0333$$

$$P_2(1.0) = e^{-1.0} \frac{(1.0)^2}{2!} \approx 0.1839$$

$$\frac{P_2(\mu)}{P_2(\mu')} \cdot \frac{\mu'}{\mu} = \frac{0.0333}{0.1839} \cdot \frac{1.0}{0.3} \approx 0.604 < 1 \quad \checkmark$$

---

In [2]:
# Demo script for PNS attack detection using enhanced DecoyStateQKD

# Create QKD system with carefully chosen parameters
# Following Hwang's example: μ_signal = 0.3, μ_decoy = 1.0
qkd = DecoyStateQKD(
    mu_signal=0.3,                    # Signal intensity (mostly single photons)
    mu_decoy=[0.7, 0.05],            # Strong decoy + weak decoy
    decoy_probabilities=[0.3, 0.2],  # 30% strong decoy, 20% weak decoy, 50% signal
    detection_efficiency=1.0,         # Perfect detectors for clarity
    technical_error_rate=0.01,        # Low technical error
    random_seed=1337
)

print("🔐 Enhanced BB84 with PNS Attack Detection")
print("=" * 50)
print("Implementation of Hwang's 2003 Decoy State Method")
print()

# Test with moderate number of pulses for clear demonstration
bits_length = 5000000
print(f"Testing with {bits_length:,} pulses")
print()

# Run security comparison
comparison_results = qkd.run_security_comparison(bits_length=bits_length, verbose=True)

print("\n" + "=" * 60)
print("SECURITY ANALYSIS SUMMARY")
print("=" * 60)

for scenario, result in comparison_results.items():
    print(f"\n{scenario.replace('_', ' ').title()}:")
    print(f"  Status: {'✅ SECURE' if result.is_secure else '⚠️  COMPROMISED'}")
    print(f"  Signal yield (Y_s): {result.signal_yield:.6f}")
    print(f"  Security margin: {result.security_margin:.6f}")
    print(f"  Estimated multiphoton yield bound: {result.estimated_multiphoton_yield:.6f}")

# Detailed mathematical verification
print("\n" + "=" * 60)
print("MATHEMATICAL VERIFICATION")
print("=" * 60)

clean_result = comparison_results['clean_channel']
attack_result = comparison_results['pns_attack']

print(f"\nHwang's Security Condition (Eq. 12): Y_s > (P_2(μ)/P_2(μ')) * Y_d")
print(f"Where:")
print(f"  μ_signal = {qkd.mu_signal}")
print(f"  μ_decoy = {qkd.mu_decoy[0]}")
print(f"  P_2(μ_signal) = {clean_result.analysis_details['P_2_signal']:.6f}")
print(f"  P_2(μ_decoy) = {clean_result.analysis_details['P_2_decoy']:.6f}")
print(f"  Security ratio = {clean_result.analysis_details['security_ratio']:.6f}")

print(f"\nClean Channel:")
print(f"  {clean_result.signal_yield:.6f} > {clean_result.security_condition_rhs:.6f} ? {clean_result.is_secure}")

print(f"\nWith PNS Attack:")
print(f"  {attack_result.signal_yield:.6f} > {attack_result.security_condition_rhs:.6f} ? {attack_result.is_secure}")

# Show the attack detection capability
if clean_result.is_secure and not attack_result.is_secure:
    print(f"\n🎯 SUCCESS: Decoy state method successfully detected PNS attack!")
    print(f"   Clean channel passed security test")
    print(f"   Attacked channel failed security test")
elif not clean_result.is_secure:
    print(f"\n⚠️  Note: Even clean channel appears insecure with current parameters")
    print(f"   This suggests either too high loss or suboptimal parameter choice")
else:
    print(f"\n📊 Both scenarios appear secure with current parameters")
    print(f"   Attack may not be strong enough or parameters need adjustment")

print(f"\nDecoy yields comparison:")
for label in clean_result.decoy_yields:
    clean_yield = clean_result.decoy_yields[label]
    attack_yield = attack_result.decoy_yields[label]
    ratio = attack_yield / clean_yield if clean_yield > 0 else float('inf')
    print(f"  {label}: {clean_yield:.6f} → {attack_yield:.6f} (ratio: {ratio:.2f})")

print(f"\n" + "=" * 60)
print("PROTOCOL IMPLEMENTATION NOTES")
print("=" * 60)
print("✅ Poisson photon statistics: P_n(μ) = e^(-μ) * μ^n / n!")
print("✅ Multi-intensity decoy states for security analysis")
print("✅ Realistic detection model with efficiency losses")
print("✅ PNS attack simulation (Eve blocks n=1, steals from n≥2)")
print("✅ Yield-based security condition (independent of channel loss)")
print("✅ Mathematical verification of Hwang's 2003 security proof")

🔐 Enhanced BB84 with PNS Attack Detection
Implementation of Hwang's 2003 Decoy State Method

Testing with 5,000,000 pulses

=== Security Comparison: Clean Channel vs PNS Attack ===

--- Clean Channel ---


Bob measuring: 100%|██████████| 5000000/5000000 [00:03<00:00, 1444457.94it/s]


=== PNS Attack Detection Analysis ===
Based on Hwang's 2003 decoy state method

Measured yields:
  Y_s (signal): 0.2593
  Y_d (decoy_1): 0.0484
  Y_d (decoy_0): 0.5032

Poisson probabilities (2-photon):
  P_2(μ_s=0.300) = 0.033337
  P_2(μ_d=0.700) = 0.121663

Security Analysis (Hwang's Eq. 12):
  Security condition: Y_s > (P_2(μ)/P_2(μ')) * Y_d
  Left side (Y_s): 0.259253
  Right side: (0.033337/0.121663) * 0.503217 = 0.137886
  Security margin: 0.121367
  Channel status: SECURE
  ✅ Channel appears secure against PNS attacks

--- Pns Attack ---


Bob measuring: 100%|██████████| 5000000/5000000 [00:02<00:00, 1969236.50it/s]


=== PNS Attack Detection Analysis ===
Based on Hwang's 2003 decoy state method

Measured yields:
  Y_s (signal): 0.0368
  Y_d (decoy_0): 0.1555
  Y_d (decoy_1): 0.0013

Poisson probabilities (2-photon):
  P_2(μ_s=0.300) = 0.033337
  P_2(μ_d=0.700) = 0.121663

Security Analysis (Hwang's Eq. 12):
  Security condition: Y_s > (P_2(μ)/P_2(μ')) * Y_d
  Left side (Y_s): 0.036788
  Right side: (0.033337/0.121663) * 0.155513 = 0.042612
  Security margin: -0.005824
  Channel status: COMPROMISED
  ⚠️  WARNING: Possible PNS attack detected!
     Decoy yield is abnormally high compared to signal yield

SECURITY ANALYSIS SUMMARY

Clean Channel:
  Status: ✅ SECURE
  Signal yield (Y_s): 0.259253
  Security margin: 0.121367
  Estimated multiphoton yield bound: 0.137886

Pns Attack:
  Status: ⚠️  COMPROMISED
  Signal yield (Y_s): 0.036788
  Security margin: -0.005824
  Estimated multiphoton yield bound: 0.042612

MATHEMATICAL VERIFICATION

Hwang's Security Condition (Eq. 12): Y_s > (P_2(μ)/P_2(μ')) * Y_